In [5]:
import requests
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import weather_plots as wp 

class OpenWeatherAPI:
    api_key =None
    current_data = None
    forecast_data = None
    current_df = None
    forecast_df = None
    def __init__(self, api_key):
        self.api_key = api_key
    def request(self, url):
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()
    def fetch_weather_data(self, location):
        self.current_data = self.request(f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={self.api_key}")
        self.current_df = {
                "Location": self.current_data["name"],
                "Temperature (C)": self.current_data["main"]["temp"]-273.15,
                "Humidity (%)": self.current_data["main"]["humidity"],
                "Weather": self.current_data["weather"][0]["description"]
        }
        self.current_df= pd.DataFrame([self.current_df])
        self.forecast_data = self.request(f"http://api.openweathermap.org/data/2.5/forecast?q={location}&appid={self.api_key}")
        temp = []
        for entry in self.forecast_data["list"]:
            temp.append({
                'Datetime': pd.to_datetime(entry['dt'], unit='s'),
                'Temperature (C)': entry['main']['temp'] - 273.15,
                'Humidity (%)': entry['main']['humidity'],
                'Wind Speed (m/s)': entry['wind']['speed'],
                'Weather': entry['weather'][0]['description']
            })
        self.forecast_df = pd.DataFrame(temp)
        return self.current_df, self.forecast_df

def calculate_daily_stats(df_forecast):
    
    # Extract the date from the 'Datetime' column and create a new 'Date' column
    df_forecast['Date'] = df_forecast['Datetime'].dt.date

    # Group the data by the 'Date' column and calculate the min and max for each group
    daily_stats = df_forecast.groupby('Date').agg({
        'Temperature (C)': ['min', 'max'],
        'Humidity (%)': ['min', 'max'],
        'Wind Speed (m/s)': ['min', 'max']
    })

    # Flatten the MultiIndex columns
    daily_stats.columns = ['Min Temperature (C)', 'Max Temperature (C)', 
                           'Min Humidity (%)', 'Max Humidity (%)', 
                           'Min Wind Speed (m/s)', 'Max Wind Speed (m/s)']
    
    # Reset the index to turn the 'Date' back into a column
    daily_stats.reset_index(inplace=True)
    
    return daily_stats

def determine_weather(row):
    if row['Max Temperature (C)'] > 25:
        return 'Sunny'
    elif row['Min Temperature (C)'] < 15 and row['Max Humidity (%)'] > 80:
        return 'Rainy'
    else:
        return 'Partly Cloudy'

def show(df):
    x_ticks = df['Datetime'][::6]

    fig, axs = plt.subplots(2, 2, figsize=(14, 10))

    # Plot 1: Line plot for Temperature over Time
    axs[0, 0].plot(df['Datetime'], df['Temperature (C)'], marker='o')
    axs[0, 0].set_title('Temperature over Time')
    axs[0, 0].set_xlabel('Datetime')
    axs[0, 0].set_ylabel('Temperature (C)')
    axs[0, 0].set_xticks(x_ticks)
    axs[0, 0].tick_params(axis='x', rotation=45)
    max_temp = df['Temperature (C)'].max()
    max_temp_time = df.loc[df['Temperature (C)'] == max_temp, 'Datetime'].iloc[0]
    axs[0, 0].annotate(f'Max Temp: {max_temp:.2f}C', xy=(max_temp_time, max_temp), 
                xytext=(max_temp_time, max_temp+2), 
                arrowprops=dict(facecolor='black', shrink=0.05))
    axs[0, 0].grid(True, linestyle='--', linewidth=0.5)

    # # Add line of best fit for Temperature over Time
    s = np.polyfit(df['Datetime'].astype(np.int64) // 10**9, df['Temperature (C)'], 1)
    p = np.poly1d(s)
    axs[0, 0].plot(df['Datetime'], p(df['Datetime'].astype(np.int64) // 10**9), "r--")

    # # Plot 2: Bar plot for Humidity over Time
    axs[0, 1].bar(df['Datetime'], df['Humidity (%)'], color='skyblue')
    axs[0, 1].set_title('Humidity over Time')
    axs[0, 1].set_xlabel('Datetime')
    axs[0, 1].set_ylabel('Humidity (%)')
    axs[0, 1].set_xticks(x_ticks)
    axs[0, 1].tick_params(axis='x', rotation=45)

    # # Plot 3: Pie chart for Weather Description
    weather_counts = df['Weather'].value_counts()
    axs[1, 0].pie(weather_counts, labels=weather_counts.index, autopct='%1.1f%%', startangle=140)
    axs[1, 0].set_title('Weather Description Distribution')
    axs[1, 0].axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

    # # Plot 4: Scatter plot for Temperature vs. Wind Speed
    axs[1, 1].scatter(df['Temperature (C)'], df['Wind Speed (m/s)'], c='g', marker='o')
    axs[1, 1].set_title('Temperature vs. Wind Speed')
    axs[1, 1].set_xlabel('Temperature (C)')
    axs[1, 1].set_ylabel('Wind Speed (m/s)')
    axs[1, 1].grid(True)

    # # Add line of best fit for Temperature vs. Wind Speed
    s = np.polyfit(df['Temperature (C)'], df['Wind Speed (m/s)'], 1)
    p = np.poly1d(s)
    axs[1, 1].plot(df['Temperature (C)'], p(df['Temperature (C)']), "r--")

    # # Adjust layout
    plt.tight_layout()
    plt.show()

city_dropdown = widgets.Dropdown(
    options=['Sydney', 'New York', 'London', 'Beijing'],
    value='Sydney',
    description='City:'
)
# display(city_dropdown)
output = widgets.Output()
display(output)
update_button = widgets.Button(
    description='Update Weather',
    button_style='success'
)
# display(update_button)
output = widgets.Output()
# display(output)

# Function to update weather data and plots
def update_weather(b):
    city = city_dropdown.value
    api = OpenWeatherAPI('2dd249caf078aec871ab451b46fee0a1')
    current_df, daily_df = api.fetch_weather_data(city)
    daily_df = calculate_daily_stats(daily_df)
    daily_df['Weather'] = daily_df.apply(determine_weather, axis=1)

    with output:
        output.clear_output()
        fig, axs = plt.subplots(1, 2, figsize=(15, 4))
        fig.tight_layout(pad=5.0)

        # Generate plots
        wp.create_forecast_card(axs[0], daily_df.iloc[0]['Date'], daily_df.iloc[0]['Max Temperature (C)'], daily_df.iloc[0]['Min Temperature (C)'], daily_df.iloc[0]['Weather'])
        # wp.plot_scatter(hourly_df, 'Temperature (C)', 'Wind Speed (m/s)', axs[1], title="Wind Speed vs Temperature")
        wp.plot_min_max(daily_df, 'Date', 'Min Temperature (C)', 'Max Temperature (C)', axs[1], title="Daily Min/Max Temperature")

        plt.show()

update_button.on_click(update_weather)
display(city_dropdown, update_button, output)
update_weather(None)

Output()

Dropdown(description='City:', options=('Sydney', 'New York', 'London', 'Beijing'), value='Sydney')

Button(button_style='success', description='Update Weather', style=ButtonStyle())

Output()